In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict

In [30]:
reviews_raw = open('yelp_academic_dataset_review.json', 'r')


counter = 0

data_dict = defaultdict(list)

for line in reviews_raw:
    if counter >= 3:
        break
    #print line
    line_json = json.loads(line)
    
    user_id = line_json['user_id']
    review_type = line_json['type']
    stars = line_json['stars']
    date = line_json['date']
    business_id = line_json['business_id']
    text = line_json['text']
    
    data_dict['user_id'].append(user_id)
    data_dict['stars'].append(stars)
    data_dict['business_id'].append(business_id)
    data_dict['text'].append(text)
    
    
    #print user_id, business_id, stars
    #counter += 1

#print dict(data_dict)
    
reviews_raw.close() 

df = pd.DataFrame(data_dict)
df.head()


,business_id,stars,text,user_id
0,vcNAWiLM4dR7D2nwwJ7nCA,5,dr. goldberg offers everything i look for in a...,Xqd0DzHaiyRqVH3WRG7hzg
1,vcNAWiLM4dR7D2nwwJ7nCA,2,"Unfortunately, the frustration of being Dr. Go...",H1kH6QZV7Le4zqTRNxoZow
2,vcNAWiLM4dR7D2nwwJ7nCA,4,Dr. Goldberg has been my doctor for years and ...,zvJCcrpm2yOZrxKffwGQLA
3,vcNAWiLM4dR7D2nwwJ7nCA,4,Been going to Dr. Goldberg for over 10 years. ...,KBLW4wJA_fwoWmMhiHRVOA
4,vcNAWiLM4dR7D2nwwJ7nCA,4,Got a letter in the mail last week that said D...,zvJCcrpm2yOZrxKffwGQLA


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1569264 entries, 0 to 1569263
Data columns (total 4 columns):
business_id    1569264 non-null object
stars          1569264 non-null int64
text           1569264 non-null object
user_id        1569264 non-null object
dtypes: int64(1), object(3)
memory usage: 59.9+ MB


In [26]:
businesses_raw = open('yelp_academic_dataset_business.json', 'r')

#this table has different fields in different columns 
#extract all the possible values from the data? 
#get all possible values of attributes, ambience, and categories 
#there are multiple levels of nesting... 

fields = set()
fields_amb = set()
fields_cat = set()

counter = 1

for line in businesses_raw:
        
    if counter >= 6:
        break
    line_json = json.loads(line)
    
    if 'Restaurants' not in line_json['categories']:
        continue
        
    #print line_json
    
    #attributes
    attributes = line_json['attributes']
    for key in attributes:
        #print key
        fields.add(key)
        
    #ambience (not always present in attributes so we need a try/except)
    try:
        ambience = line_json['attributes']['Ambience']
        for key in ambience:
            fields_amb.add(key)
    except(KeyError): 
        pass
    
    #counter to only test for a few rows
    #counter += 1
 
    #categories
    categories = line_json['categories']
    for key in categories:
        #print key
        fields_cat.add(key)    

print fields
#print line_json
print fields_amb
print fields_cat
    

    

set([u'Drive-Thru', u'Alcohol', u'Open 24 Hours', u'Noise Level', u'Has TV', u'Attire', u'Ambience', u'Good for Kids', u'Price Range', u'BYOB', u'Accepts Insurance', u'Good For Dancing', u'Delivery', u'Dogs Allowed', u'Coat Check', u'Payment Types', u'Smoking', u'Accepts Credit Cards', u'BYOB/Corkage', u'Take-out', u'Ages Allowed', u'Corkage', u'By Appointment Only', u'Happy Hour', u'Wheelchair Accessible', u'Outdoor Seating', u'Takes Reservations', u'Waiter Service', u'Wi-Fi', u'Dietary Restrictions', u'Caters', u'Order at Counter', u'Good For', u'Parking', u'Music', u'Good For Kids', u'Good For Groups'])
set([u'romantic', u'intimate', u'classy', u'hipster', u'divey', u'touristy', u'trendy', u'upscale', u'casual'])
set([u'Swiss Food', u'Salvadoran', u'Drugstores', u'Dry Cleaning & Laundry', u'Tapas/Small Plates', u'Canadian (New)', u'Fondue', u'Buffets', u'Gluten-Free', u'Arts & Entertainment', u'Arabian', u'Oriental', u'Sandwiches', u'Pets', u'Austrian', u'Shaved Ice', u'Creperies', 

In [34]:

businesses_raw = open('yelp_academic_dataset_business.json', 'r')

counter = 0

#explode the attributes and categories 

business_dict = defaultdict(list)

for line in businesses_raw:
    if counter >= 10:
        break
   
    #print line
    line_json = json.loads(line)
    
    #only keep restaurants
    if 'Restaurants' not in line_json['categories']:
        continue
    
    #only keep businesses with reviews >= 10 
    if line_json['review_count'] < 10:
        continue
    
    city = line_json['city']
    bus_stars = line_json['stars']
    review_count = line_json['review_count']
    categories = line_json['categories']
    attributes = line_json['attributes']
    name = line_json['name']
    
    business_dict['city'].append(city)
    business_dict['bus_stars'].append(bus_stars)  
    business_dict['review_count'].append(review_count) 
    business_dict['categories'].append(categories) 
    business_dict['attributes'].append(attributes) 
    business_dict['name'].append(name) 

    #get all the attributes as separate columns
    for attribute in fields:
        try:
            value = line_json['attributes'][attribute]
        except(KeyError):
            value = ''
        business_dict['atr_' + attribute].append(value) 
    
    for ambience in fields_amb:
        try:
            value = line_json['attributes']['Ambience'][ambience]
        except(KeyError):
            value = ''
        business_dict['amb_' + ambience].append(value) 
 
    #get the ambience out of the attributes
    
    for category in fields_cat:
        if category in line_json['categories']:
            business_dict['cat_' + category].append(True) 
        else:
            business_dict['cat_' + category].append(False) 

    
    #counter to only test for a few rows
    #counter += 1

#print dict(data_dict)

#TFIDF for the attributes??  


businesses_raw.close() 

df_bus = pd.DataFrame(business_dict)


df_bus.head().to_csv('df_bus_out.csv')

print len(business_dict)
df_bus.head()

#try using the difference between the business rating and hte user rating to determine how much a user relatively likes it? 

336


,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale,atr_Accepts Credit Cards,...,cat_Vietnamese,cat_Wedding Planning,cat_Wine Bars,cat_Wineries,cat_Wok,cat_Yoga,categories,city,name,review_count
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,Emil's Lounge,11
1,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,"[Bars, American (Traditional), Nightlife, Rest...",Carnegie,Alexion's Bar & Grill,15
2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,"[Breakfast & Brunch, Restaurants]",Carnegie,Gab & Eat,38
3,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,"[Restaurants, Italian]",Carnegie,Papa J's,46
4,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,"[Burgers, Fast Food, Restaurants]",Munhall,Steak 'n Shake,36


In [29]:
df_bus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14023 entries, 0 to 14022
Columns: 336 entries, amb_casual to review_count
dtypes: bool(284), float64(1), int64(1), object(50)
memory usage: 9.5+ MB


In [ ]:
#replace all true/false with 0/1

In [ ]:
#join with the user review data